# Programmieraufgabe: 3D-Printer

Eine kleine Firma betreibt professionellen 3D-Druck. In der Woche kann die Firma den 3D-Drucker $T$ Stunden betreiben. Die Aufträge $A_1,\ldots, A_n$ werden der Firma angeboten. Die Firma will gewisse dieser Aufträge auswählen und die anderen ablehnen (oder auf später verschieben), und so den Wochengewinn maximieren. Jeder Auftrag $A_i$ ist durch eine Bearbeitungsdauer $d_i$ (in Stunden) und einen Gewinn $p_i$ (in Euro) beschrieben. Dazu kommen gewisse technische Unterschiede in den Aufträgen, die gegebenfalls eine Reinigung oder einen Umbau des 3D-Druckers bedingen. Wir werden diese Unterschiede der Einfachheit unter dem Stichwort Substrat $s_i$ zusammenfassen: Ein Auftrag braucht vielleicht nur schnödes graues Plastik, ein anderer eine Plastik-Metallmischung in orange. (Ich habe keine Ahnung wie 3D-Drucker funktionieren.)


### Aufgabe (a): Einfaches MIP
Implementieren Sie ein MIP und lösen Sie <code>simple_instance</code>. Führen Sie dazu Indikatorvariablen <code>x[i]</code> ein, die angeben, ob der Auftrag <code>i</code> angenommen wird oder nicht. Führen Sie zusätzlich eine Variable <code>profit</code> ein, die den Gewinn beziffert. Nutzen Sie den Code unten, um die angenommenen Aufträge und den Gewinn auszugeben.

In [ ]:
def simple_instance():
    T = 30
    d = [2,3,6,3,4,7,4,2,4,5]
    p = [22,31,65,33,42,71,41,22,30,59]
    return T,d,p

Wie immer muss das Paket <code>mip</code> installiert sein. In Colab (oder auf Ihrem eigenen Rechner) installieren Sie <code>mip</code> gegebenfalls durch Löschen der Raute unten. 

In [ ]:
# Raute Löschen, um mip zu installieren
# !pip install mip
import mip

### Lösung (a)

In [ ]:
T,d,p=simple_instance()
n=len(d)

### Fügen Sie Ihren Code hier ein ###

Schließlich die Ausgabe der Lösung.

In [ ]:
accepted_jobs=[i for i in range(n) if x[i].x>0]
print("Angenommene Aufträge: {}".format(accepted_jobs))
print("Gewinn: {}EUR".format(profit.x))

## MIP mit Rüstzeiten
Das Modell bisher war sehr einfach. In der Realität kommen oft Rüstzeiten dazu: Der 3D-Drucker muss umgerüstet, eventuell gereinigt werden, wenn sich das Substrat ändert. Bestand etwa bei einem Auftrag das Substrat aus grauem Plastik und soll nun der folgende Auftrag mit weißem Plastik durchgeführt werden, so muss der Drucker gereinigt werden, um Farbfehler zu vermeiden. Wie viel Zeit auf die Umrüstung / Reinigung verwendet werden muss, hängt dabei davon ab, von welchem Substrat auf welches Substrat gewechselt wird. Konkret: Im Gegensatz zu vorher muss nun eine Reihenfolge der angenommenen Aufträge bestimmt werden, und wenn dann auf einen Auftrag $A_i$ mit Substrat $k=s_i$ der Auftrag $A_j$ mit $\ell=s_j$ folgt, wird eine Rüstzeit von $r_{k,\ell}$ nötig.

Es sei $\{1,\ldots, C\}$ die Menge der unterschiedlichen Substrate. Für die Rüstzeiten soll gelten:
* $r_{k,\ell}\geq 0$ für alle $k,\ell\in \{1,\ldots, C\}$, dh, keine Reisen in die Vergangenheit.
* $r_{k,k}=0$ für all $k\in\{1,\ldots, C\}$, dh wenn das Substrat nicht gewechselt wird, fallen keine Rüstzeiten an.
* es gilt eine Dreiecksungleichung $r_{k,m}\leq r_{k,\ell}+r_{\ell,m}$ für alle $k,\ell,m\in\{1,\ldots, C\}$, dh, Rüstzeiten können nicht verkürzt werden, in dem auf ein drittes Substrat gewechselt wird. (Ist ja klar.)

Es kann aber durchaus $r_{k,\ell}\neq r_{\ell,k}$ gelten, dh, die Rüstzeiten sind nicht notwendig symmetrisch. Der Wechsel von weißem Substrat auf graues erfordert eventuell eine kürzere Reinigung als umgekehrt.

Am Ende ist keine Reinigung nötig. (Ich weiß, igitt.)

### Aufgabe (b): Reihenfolge 
Argumentieren Sie, dass alle (angenommenen) Aufträge mit gleichem Substrat konsekutiv ausgeführt werden können. Dh, es reicht aus, die Substrate der angenommenen Aufträge in eine Reihenfolge zu bringen. Schreiben Sie Ihre Argumentation in die folgende *markdown*-Zelle.

### Lösung (b)

*hier Ihre Argumentation*

### Aufgabe (c): MIP mit Rüstzeiten
Implementieren Sie das MIP mit Rüstzeiten und lösen Sie <code>small_instance</code> oder <code>large_instance</code>. 

Anleitung: Wie oben führen Sie Variablen <code>x[i]</code> und <code>profit</code> ein. Führen Sie zusätzlich für $k,p=1,\ldots,C$ eine Indikatorvariable <code>z[k][p]</code> ein, die bestimmt, ob das Substrat $k$ an $p$ter Stelle der Reihung der Substrate stehen soll. Die Variable <code>R</code> soll die Gesamtrüstzeit bezeichnen. Nutzen Sie den Code unten, um die angenommenen Aufträge, die Reihung der Substrate, die Gesamtrüstzeit und den Gewinn auszugeben. Sie werden sicher noch weitere Variablen einführen müssen.

In [ ]:
def small_instance():
    T,d,p=simple_instance()
    s = [1, 2, 0, 2, 3, 0, 3, 1, 3, 4]
    r = [[0, 6, 8, 7, 7],
         [6, 0, 5, 4, 4],
         [5, 5, 0, 4, 4],
         [4, 4, 6, 0, 7],
         [8, 6, 7, 7, 0]]
    C=max(s)+1
    return T,d,p,s,r,C

def large_instance():
    T = 60
    d = [2,3,6,3,4,7,4,2,4,5,6,2,5,7,2,3,4,3,7,8,6,6,2,3,5,9,6,7,4,1]
    p = [22,31,65,33,42,71,41,22,30,59,63,22,51,72,27,38,46,36,77,84,64,64,23,32,53,91,61,71,43,15]
    s = [1, 2, 0, 2, 3, 0, 3, 1, 3, 4, 1, 1, 4, 0, 1, 2, 3, 2, 0, 0, 4, 4, 3, 2, 1, 0, 1, 2, 3, 0]
    r = [[0, 6, 8, 7, 7],
         [6, 0, 5, 4, 4],
         [5, 5, 0, 4, 4],
         [4, 4, 6, 0, 7],
         [8, 6, 7, 7, 0]]
    C=max(s)+1
    return T,d,p,s,r,C

### Lösung (c)



In [ ]:
T,d,p,s,r,C=small_instance()
#T,d,p,s,r,C=large_instance()
n=len(d)

### Fügen Sie Ihren Code hier ein ###

Ausgabe der Lösung

In [ ]:
sequence=[k for p in range(C) for k in range(C) if z[k][p].x>0] 
jobs=[i for i in range(n) if x[i].x>0]
setup_time=R.x
profit_opt=profit.x
print("Angenommene Aufträge: {}".format(jobs))
print("Reihenfolge der Substrate: {}".format(sequence))
print("Gesamtrüstzeit: {}".format(setup_time))
print("Gewinn: {}EUR".format(profit_opt))